In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle
import os

In [2]:
data_dir = "../data"
images_path = os.path.join(data_dir, "images")
masks_path = os.path.join(data_dir, "masks")
processed_images_path = os.path.join(data_dir, "processed_data2/images")
processed_masks_path = os.path.join(data_dir, "processed_data2/masks")
augmented_images_path = os.path.join(data_dir, "processed_data2/augmented_images")
augmented_masks_path = os.path.join(data_dir, "processed_data2/augmented_masks")
models_path = "../models2"

In [ ]:
os.makedirs(processed_images_path, exist_ok=True)
os.makedirs(processed_masks_path, exist_ok=True)

In [10]:
os.makedirs(augmented_images_path, exist_ok=True)
os.makedirs(augmented_masks_path, exist_ok=True)

In [20]:
os.makedirs(models_path, exist_ok=True)

In [3]:
def preprocess_data(image_dir, mask_dir, output_image_dir, output_mask_dir, target_size=(256, 256)):
    """
    Bu fonksiyon:
    - Yüksek çözünürlüklü görselleri 256x256'ye küçültür.
    - Maskeleri normalize eder ve kaydeder.
    """
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith(".png")]
    
    for img_file in tqdm(image_files, desc="Processing Images and Masks"):
        mask_file = img_file.replace(".jpg", "_segmentation.png")
        if mask_file not in mask_files:
            continue  
        
        img = cv2.imread(os.path.join(image_dir, img_file))
        img = cv2.resize(img, target_size)
        
        mask = cv2.imread(os.path.join(mask_dir, mask_file), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, target_size)
        mask = mask / 255.0  
        
        cv2.imwrite(os.path.join(output_image_dir, img_file), img)
        cv2.imwrite(os.path.join(output_mask_dir, mask_file), (mask * 255).astype(np.uint8))


In [ ]:
preprocess_data(images_path, masks_path, processed_images_path, processed_masks_path)
print("Preprocessing tamamlandı. İşlenmiş veriler kaydedildi.")

Processing Images and Masks: 100%|██████████| 2594/2594 [03:12<00:00, 13.47it/s]

Preprocessing tamamlandı. İşlenmiş veriler kaydedildi.


In [14]:
def augment_data(image_dir, mask_dir, output_image_dir, output_mask_dir):
    """
    Görseller ve maskelere augmentation uygular: döndürme, çevirme, parlaklık.
    """
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith(".png")]
    
    for img_file in tqdm(image_files, desc="Augmenting Images and Masks"):
        mask_file = img_file.replace(".jpg", "_segmentation.png")
        if mask_file not in mask_files:
            continue
        
        img = cv2.imread(os.path.join(image_dir, img_file))
        mask = cv2.imread(os.path.join(mask_dir, mask_file), cv2.IMREAD_GRAYSCALE)
        
        img = tf.convert_to_tensor(img, dtype=tf.float32)
        img = tf.expand_dims(img, axis=-1) if len(img.shape) == 2 else img
        mask = tf.convert_to_tensor(mask, dtype=tf.float32)
        mask = tf.expand_dims(mask, axis=-1)
        
        for i in range(3): 
            augmented_img = tf.image.random_flip_left_right(img)
            augmented_img = tf.image.random_flip_up_down(augmented_img)
            augmented_img = tf.image.random_brightness(augmented_img, max_delta=0.1)
            
            augmented_mask = tf.image.random_flip_left_right(mask)
            augmented_mask = tf.image.random_flip_up_down(augmented_mask)
            
            aug_img_name = f"aug_{i}_{img_file}"
            aug_mask_name = f"aug_{i}_{mask_file}"
            augmented_img = tf.squeeze(augmented_img).numpy().astype(np.uint8)
            augmented_mask = tf.squeeze(augmented_mask).numpy().astype(np.uint8)
            
            cv2.imwrite(os.path.join(output_image_dir, aug_img_name), augmented_img)
            cv2.imwrite(os.path.join(output_mask_dir, aug_mask_name), augmented_mask)


In [15]:
augment_data(processed_images_path, processed_masks_path, augmented_images_path, augmented_masks_path)
print("Data augmentation tamamlandı. Augmented veriler kaydedildi.")

Augmenting Images and Masks: 100%|██████████| 2594/2594 [00:49<00:00, 51.91it/s]

Data augmentation tamamlandı. Augmented veriler kaydedildi.


In [11]:
def load_data(image_dir, mask_dir):
    images = []
    masks = []
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith(".png")]
    
    for img_file in tqdm(image_files, desc="Loading Data"):
        mask_file = img_file.replace(".jpg", "_segmentation.png")
        if mask_file in mask_files:
            img = cv2.imread(os.path.join(image_dir, img_file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0
            mask = cv2.imread(os.path.join(mask_dir, mask_file), cv2.IMREAD_GRAYSCALE) / 255.0
            images.append(img)
            masks.append(mask)
    
    images = np.array(images, dtype=np.float32)
    masks = np.expand_dims(np.array(masks, dtype=np.float32), axis=-1)
    return images, masks


In [3]:
def preprocess_and_save_as_npy(image_dir, mask_dir, output_image_dir, output_mask_dir, target_size=(256, 256)):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)

    image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith(".png")]

    for img_file in tqdm(image_files, desc="Preprocessing and Saving as NPY"):
        mask_file = img_file.replace(".jpg", "_segmentation.png")
        if mask_file in mask_files:
            img = cv2.imread(os.path.join(image_dir, img_file))
            img = cv2.resize(img, target_size) / 255.0

            mask = cv2.imread(os.path.join(mask_dir, mask_file), cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, target_size) / 255.0

            np.save(os.path.join(output_image_dir, img_file.replace(".jpg", ".npy")), img)
            np.save(os.path.join(output_mask_dir, mask_file.replace(".png", ".npy")), mask)

In [ ]:
main_images, main_masks = load_data(processed_images_path, processed_masks_path)
aug_images, aug_masks = load_data(augmented_images_path, augmented_masks_path)

train_images = np.concatenate([main_images, aug_images], axis=0)
train_masks = np.concatenate([main_masks, aug_masks], axis=0)

Loading Data: 100%|██████████| 7782/7782 [00:12<00:00, 612.89it/s]


In [6]:
def build_unet_resnet50(input_shape=(256, 256, 3), dropout_rate=0.3, l2_lambda=0.01):
    """
    ResNet50 encoder ve UNET decoder kullanarak model oluşturur.
    """
    resnet = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    resnet.trainable = False  

    encoder_output = resnet.get_layer("conv4_block6_out").output
    
    x = UpSampling2D(size=(2, 2))(encoder_output)
    x = Conv2D(256, (3, 3), activation="relu", padding="same", kernel_regularizer=l2(l2_lambda))(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation="relu", padding="same", kernel_regularizer=l2(l2_lambda))(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation="relu", padding="same", kernel_regularizer=l2(l2_lambda))(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(32, (3, 3), activation="relu", padding="same", kernel_regularizer=l2(l2_lambda))(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)

    output = Conv2D(1, (1, 1), activation="sigmoid", padding="same")(x)

    model = Model(inputs=resnet.input, outputs=output)
    return model

In [7]:
model = build_unet_resnet50()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="binary_crossentropy", metrics=["accuracy"])

2024-12-16 16:48:44.165181: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-12-16 16:48:44.166195: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-16 16:48:44.166867: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-16 16:48:44.167372: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-16 16:48:44.167832: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(os.path.join(models_path, 'unet_resnet50_best.keras'), save_best_only=True)

In [ ]:
history = model.fit(train_images, train_masks,
                    validation_split=0.2,
                    batch_size=8,
                    epochs=50,
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/50


2024-12-16 16:49:28.309684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/tensorflow-metal/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


KeyboardInterrupt: 